**공부한 자료: Machine Learning with Python Cookbook

### Transforming Features

In [1]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

features = np.array([[2,3],
                   [2,3],
                   [2,3]])

In [2]:
def mul_ten(x):
    return x*10

In [3]:
ten_transformer = FunctionTransformer(mul_ten)
ten_transformer.transform(features)

array([[20, 30],
       [20, 30],
       [20, 30]])

In [5]:
# pandas의 apply함수도 이용 가능하다

import pandas as pd

df = pd.DataFrame(features, columns = ['feature_1', 'feature_2'])
df

,feature_1,feature_2
0,2,3
1,2,3
2,2,3


In [6]:
df = df.apply(mul_ten)
df

,feature_1,feature_2
0,20,30
1,20,30
2,20,30


이렇게 사이킷런에서 제공하는 FunctionTransformer나 판다스에서 제공하는 apply()를 이용하면 숫자형데이터를 원하는 형태로 바꿔줄 수 있다. 

### Handling Outliers

우선, 아웃라이어를 감지해야한다. 아웃라이어를 감지하는 방법은 전체적으로 살펴보는 방법과 IQR을 사용하는 방법이 있다. 

In [7]:
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs

In [8]:
# 연습용 데이터를 만들고, 임의로 extreme values를 준다.

features, _ = make_blobs(n_samples = 10,
                        n_features = 2,
                        centers = 1,
                        random_state = 1)

features[0,0] = 10000
features[0,1] = 10000

In [10]:
features

array([[ 1.00000000e+04,  1.00000000e+04],
       [-2.76017908e+00,  5.55121358e+00],
       [-1.61734616e+00,  4.98930508e+00],
       [-5.25790464e-01,  3.30659860e+00],
       [ 8.52518583e-02,  3.64528297e+00],
       [-7.94152277e-01,  2.10495117e+00],
       [-1.34052081e+00,  4.15711949e+00],
       [-1.98197711e+00,  4.02243551e+00],
       [-2.18773166e+00,  3.33352125e+00],
       [-1.97451969e-01,  2.34634916e+00]])

In [9]:
# 감지기 생성
outlier_detector = EllipticEnvelope(contamination=.1)
# 훈련
outlier_detector.fit(features)
# 예측
outlier_detector.predict(features)
#아웃라이어는 -1로, 인라이어는 1로 라벨링 해준다

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [11]:
feature = features[:,0]
feature

array([ 1.00000000e+04, -2.76017908e+00, -1.61734616e+00, -5.25790464e-01,
        8.52518583e-02, -7.94152277e-01, -1.34052081e+00, -1.98197711e+00,
       -2.18773166e+00, -1.97451969e-01])

In [13]:
# IQR 값을 통해  아웃라이어의 인덱스를 리턴

def indicies_of_outliers(x):
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower_fence = q1 - (iqr*1.5)
    upper_fence = q3 + (iqr*1.5)
    return np.where((x > upper_fence) | (x < lower_fence))

indicies_of_outliers(feature)

(array([0], dtype=int64),)

아웃라이어를 다루는 방법은 크게 3가지가 있다.  
1. 아웃라이어를 포함한 행을 드랍한다
2. 아웃라이어를 포함하고 있다는 표시를 한다
3. 데이터를 변환해준다(ex. log(x))

- 사이킷런 RobustScaler 설명: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler

In [15]:
# 3. 데이터 변환 - RobustScaler 이용
'''
통계학에서 로버스트한 통계량은 이상치/에러값으로 부터 영향을
크게 받지 않는 통계량을 의미한다.
'''

from sklearn.preprocessing import RobustScaler

In [19]:
X = [[1.2233, -2.4444, 2],
    [-2.0, 1.1, 33],
    [4.2, 1.4, -2.8]]

transformer = RobustScaler().fit(X)
transformer

RobustScaler()

In [20]:
transformer.transform(X)

array([[ 0.        , -1.84392883,  0.        ],
       [-1.03977419,  0.        ,  1.73184358],
       [ 0.96022581,  0.15607117, -0.26815642]])